In [1]:
from itertools import product
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
# from torch.nn.functional import relu
# from torch.nn.functional import tanh
# from torch.nn.functional import sigmoid
from scipy.fftpack import dct
from scipy.signal import convolve2d
from skimage.measure import block_reduce
from sklearn import svm

## Reduce the data size by shrinking the image and allow symbolic dynamics to occur (based on doppler-effect).

### Load the data

In [2]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
)
data_len = {'train': len(train_data), 'test': len(test_data)}

loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),

    'test': DataLoader(test_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
}

### Symbolic dynamics through iteration and non-linearity.
Through multiple iterations apply the doppler effect to the DCT of the images and apply convolution.  
Save the results in a dataframe for easy loading.

In [30]:
def relu(x):
    return np.maximum(0, x)
relu = np.vectorize(relu)
# sigmoid = np.vectorize(lambda x: 1 / (1 + np.exp(-x)))
tanh = np.vectorize(np.tanh)

def lrelu(x):
    l_ = 0.025
    return np.maximum(0, x) + l_ * np.minimum(0, x)
lrelu = np.vectorize(lrelu)

def rrelu(x):
    return np.maximum(0, x) + np.random.rand() / 10 * np.minimum(0, x)
rrelu = np.vectorize(rrelu)

In [4]:
def apply_dynamics(sample, iters, f_act, vel, v_o, v_m, kernel, conv=False, pool=False):
    working_sample = np.copy(sample)

    for i in range(iters):
        v_s = vel(i, iters)
        # apply doppler effect to sample
        working_sample = ((v_m + v_o) / (v_m + v_s)) * working_sample  # doppler-effect
        working_sample = f_act(working_sample)  # activation function

        if conv:
            working_sample = convolve2d(working_sample, kernel, mode='valid')  # convolution

        if pool and i % 4 == 1:
            working_sample = block_reduce(working_sample, (2, 2), np.mean, cval=0.5)

    return working_sample

###### Visualization
Changing `iters` determines the size of the output (if conv and/or pooling are on).

In [10]:
samples, labels = next(iter(loaders['train']))
sample = samples[0][0]

# vel_s is negative if moving towards observer
def v1(x, n):
    return -(x + 1) / (n / 3)
def v2(x, n):
    return x
def v3(x, n):
    return 1.5

v_o = 0  # positive if moving towards source
v_m = 5.022  # small tail to avoid division errors.
kernel = np.array([[2.5, 2.5],
                   [2.5, 2.5]])

In [31]:
iters = 6
f_act = lrelu  # relu, tanh, lrelu, rrelu
conv = True
pool = True

freq_sample = dct(dct(sample.numpy().T, norm='ortho').T, norm='ortho')  # decompose sample
print(f"freq_sample {freq_sample.shape}\n{np.round(freq_sample, 2)}")
result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernel, conv=conv, pool=pool)
print(f"result {result.shape}\n{np.round(result, 2)}")

freq_sample (28, 28)
[[ 4.58 -0.54 -3.33  0.42 -0.94  0.35  0.91  0.14  0.49 -0.51 -0.23 -0.11
  -0.21  0.18  0.04  0.23 -0.04 -0.08  0.12 -0.15  0.01  0.01 -0.07  0.02
  -0.06  0.04  0.14  0.01]
 [-0.19 -1.33  0.08  2.02  0.39 -0.39 -0.48 -0.46 -0.16 -0.02  0.46  0.2
   0.02  0.03 -0.22 -0.03 -0.06 -0.06  0.1  -0.01  0.05  0.02 -0.02  0.07
   0.01 -0.06 -0.07 -0.07]
 [-2.49  0.66  1.5  -0.25  0.82 -1.15 -0.18  0.44 -0.54  0.73 -0.09 -0.28
   0.27 -0.21  0.02 -0.03  0.07  0.06 -0.13  0.1  -0.01 -0.04  0.   -0.01
   0.06 -0.02 -0.04  0.02]
 [-0.19  1.32  0.33 -1.54 -1.04 -0.44  1.55  0.58 -0.33  0.45 -0.86 -0.27
   0.29 -0.19  0.31  0.    0.03  0.11 -0.16  0.04 -0.03 -0.05  0.01 -0.04
  -0.04  0.03  0.13  0.07]
 [-1.64 -0.07  1.68 -0.77 -0.35  1.96 -0.27 -1.25 -0.21 -0.37  0.48  0.63
  -0.04  0.05 -0.13 -0.24 -0.06  0.02  0.03  0.01  0.    0.07  0.13  0.02
  -0.07 -0.07 -0.09 -0.05]
 [ 0.63  0.52 -0.69 -1.52  1.21  1.35 -2.08  0.08  1.02 -0.42  0.7  -0.37
  -0.65  0.29 -0.1   0.22  0.06

###### Convert all data
Changing `iters` determines the size of the output.

In [32]:
# iters = 18
# f_act = relu
# conv = False
# pool = False

df_results = pd.DataFrame(columns=['data', 'label', 'train'])
for dset in ["train", "test"]:
    batch_size = 100
    batches = data_len[dset] / batch_size
    print(f"Begin reducing {dset}")
    for b, (images, labels) in enumerate(loaders[dset]):
        if b == batches:
            break
        for i in range(batch_size):
            freq_sample = dct(dct(images[i].numpy()[0].T, norm='ortho').T, norm='ortho')  # decompose sample
            result = apply_dynamics(freq_sample, iters, f_act, v2, v_o, v_m, kernel, conv=conv, pool=pool)
            result_str = np.array2string(result, separator=',')
            df_results = pd.concat([df_results, pd.DataFrame([[result_str, labels[i].numpy(), dset == "train"]], columns=df_results.columns)], ignore_index=True)

        if (b + 1) % 30 == 0:
            print('Reduced [{}/{}] {} batches for {} iterations'
                  .format(b + 1, batches, dset, iters))

print(df_results.shape)
df_results.to_csv(f'freq_RedData/freq_RedData_{iters}i_conv{"_pool" if pool else ""}_{f_act.__name__}_025.csv', index=False)

Begin reducing train
Reduced [30/600.0] train batches for 6 iterations
Reduced [60/600.0] train batches for 6 iterations
Reduced [90/600.0] train batches for 6 iterations
Reduced [120/600.0] train batches for 6 iterations
Reduced [150/600.0] train batches for 6 iterations
Reduced [180/600.0] train batches for 6 iterations
Reduced [210/600.0] train batches for 6 iterations
Reduced [240/600.0] train batches for 6 iterations
Reduced [270/600.0] train batches for 6 iterations
Reduced [300/600.0] train batches for 6 iterations
Reduced [330/600.0] train batches for 6 iterations
Reduced [360/600.0] train batches for 6 iterations
Reduced [390/600.0] train batches for 6 iterations
Reduced [420/600.0] train batches for 6 iterations
Reduced [450/600.0] train batches for 6 iterations
Reduced [480/600.0] train batches for 6 iterations
Reduced [510/600.0] train batches for 6 iterations
Reduced [540/600.0] train batches for 6 iterations
Reduced [570/600.0] train batches for 6 iterations
Reduced [600/